# HW 01: Tweet Sentiment MLP, Parsing, and WSD

### This homework has three tasks:
1) Tweet Sentiment Classification with a PyTorch MLP — 7 pts
2) Transition-based dependency parser (Arc-Standard) — 5 pts
3) Lesk Word Sense Disambiguation (basic + extended + eval) — 8 pts

**BONUS:** Solve any task with an LLM — **2 pts**

#### Total: 20 points (+2 bonus)


In [ ]:
# You may run these on Colab.
!pip -q install spacy nltk torch scikit-learn
!python -m spacy download en_core_web_sm -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 95.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


## Pre-requisite code

The code in this section will be used in the tasks. **Do not change these lines.**

In [21]:

import spacy, nltk, math, random, itertools, collections
import numpy as np
import networkx as nx
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional

# NLTK resources
nltk.download("wordnet", quiet=True)
nltk.download("omw-1.4", quiet=True)
from nltk.corpus import wordnet as wn

# Load spaCy
try:
    nlp = spacy.load("en_core_web_sm")
    print("✓ spaCy model loaded")
except Exception as e:
    print("✗ spaCy model not found.", e)

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

@dataclass
class DepSentence:
    text: str
    tokens: List[str]        # 1..n tokens (index 0 is reserved for ROOT)
    pos: List[str]           # UPOS or spaCy coarse POS (len=n+1; pos[0]="ROOT")
    heads: List[int]         # heads[i] = head index of token i (0..n), heads[0] = -1
    rels: List[str]          # dependency labels (len=n+1; rels[0]="root")

def spacy_to_dep(sentence: str) -> DepSentence:
    doc = nlp(sentence)
    tokens = ["<ROOT>"] + [t.text for t in doc]
    pos     = ["ROOT"] + [t.pos_ for t in doc]
    heads   = [-1] + [ (t.head.i + 1) if t.head.i != t.i else 0 for t in doc]   # root's head=0
    rels    = ["root"] + [ t.dep_ for t in doc ]
    return DepSentence(sentence, tokens, pos, heads, rels)

def uas_las(pred_heads: List[int], gold_heads: List[int], pred_rels: Optional[List[str]] = None, gold_rels: Optional[List[str]] = None) -> Tuple[float, float]:
    """Unlabeled and labeled attachment scores excluding ROOT (index 0)."""
    assert len(pred_heads) == len(gold_heads)
    n = len(gold_heads) - 1
    correct_u, correct_l = 0, 0
    for i in range(1, n+1):
        if pred_heads[i] == gold_heads[i]:
            correct_u += 1
            if pred_rels and gold_rels and pred_rels[i] == gold_rels[i]:
                correct_l += 1
    uas = correct_u / n
    las = correct_l / n if pred_rels and gold_rels else float("nan")
    return uas, las

# A tiny toy corpus (short, mostly projective for Task 1)
RAW_SENTENCES = [
    "The quick brown fox jumps over the lazy dog.",
    "I saw the man with the telescope.",
    "Students read books in the library.",
    "She will finish the report tomorrow.",
    "Flying planes can be dangerous."
]
CORPUS = [spacy_to_dep(s) for s in RAW_SENTENCES]
print("✓ Loaded", len(CORPUS), "sentences")

def pretty_tree(tokens: List[str], heads: List[int]) -> None:
    print("TOK\tHEAD")
    for i, tok in enumerate(tokens[1:], start=1):
        print(f"{i}:{tok}\t{heads[i]}:{tokens[heads[i]] if heads[i] >= 0 else 'NA'}")

✓ spaCy model loaded
✓ Loaded 5 sentences


# TASK 1 (7 pts): Tweet Sentiment Classification with a PyTorch MLP

**High-level overview.**  
Build a **binary sentiment classifier** for tweets (positive vs. negative) using the classic **NLTK Twitter Samples**
dataset (5k positive, 5k negative). You will:
(1) preprocess with spaCy (lowercase, lemmatize, keep negations),  
(2) use a **provided Binary Bag-of-Words** vectorizer (uni/bi-grams), and  
(3) train an **MLP in PyTorch** with validation.  
Report **accuracy**, a **confusion matrix**, and show sample predictions.

**What you will build.**
- A **spaCy-based tokenizer** that removes URLs/mentions, strips `#`, keeps alphabetic tokens and **negations** (“no”, “not”, “nor”, “n’t”), removes stopwords except negations, and lemmatizes.
- A **PyTorch pipeline**: Dataset/DataLoader, MLP (Linear→ReLU→Dropout→Linear), Adam optimizer, validation split.
- An **evaluation** that prints **accuracy**, **confusion matrix**, and a few random predictions with both **sentiment word** and **label**.

---

### What is provided
- Access to the **NLTK Twitter Samples** corpus.
- The global random seed and spaCy pipeline from the pre-requisite section (reused here).
- A Binary Bag-of-Words vectorizer **already set up** for you for feature extraction and preparation.

---

### What you should implement
1) `tokenize_tweet()` — the tokenizer/lemmatizer that keeps negations and removes noise.  
2) `TweetMLP` (Linear→ReLU→Dropout→Linear) and the training loop with validation.  
3) Test-set evaluation (**accuracy**, confusion matrix) and sample predictions.

---

### Grading (7 pts)
- **1.1** Data prep & tokenizer/lemmatizer (spaCy + stopwords + negations) — **2.0 pts**  
- **1.2** PyTorch Dataset/DataLoader + MLP with Dropout — **2.0 pts**  
- **1.3** Training loop with validation — **2.0 pts**  
- **1.4** Evaluation: test accuracy — **1.0 pt**

  → Grading for this point is only for verifying that the **final reported test accuracy falls within a reasonable/expected range or value**, and **not for performing the evaluation itself.**

> **Tips**  
> • Keep negation words (“no”, “not”, “nor”, “n’t”) even if they’re stopwords.  
> • Fit the vectorizer on **train only**; use the same vectorizer to transform val/test.  
> • Note that it can take a couple of minutes for the vectorizer to transform the training data.

In [23]:
import re, random, math
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from typing import List, Tuple
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import twitter_samples, stopwords

# Ensure corpora
nltk.download("twitter_samples", quiet=True)
nltk.download("stopwords", quiet=True)

# Seed and device
try:
    RANDOM_SEED
except NameError:
    RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✓ Using device: {device}")

# Data loading & preprocessing
print("Step 1: Load tweets & build tokenizer")

# Labels: pos=1, neg=0
pos_texts = twitter_samples.strings("positive_tweets.json")
neg_texts = twitter_samples.strings("negative_tweets.json")
texts = pos_texts + neg_texts
labels = [1]*len(pos_texts) + [0]*len(neg_texts)       # binary index 1 for positive feedback
print(f"✓ Loaded {len(texts)} tweets: {len(pos_texts)} positive, {len(neg_texts)} negative")

# Stopwords (keep negations)
STOP = set(stopwords.words("english"))
NEGATIONS = {"no", "not", "nor", "n't"}
STOP_TW = STOP - NEGATIONS

# Reuse spaCy pipeline if available; otherwise load it
try:
    nlp
except NameError:
    import spacy
    nlp = spacy.load("en_core_web_sm")

def tokenize_tweet(text: str, use_lemmas: bool = True) -> List[str]:
    """
    spaCy-based tokenizer for tweets.
    - Removes URLs and @mentions
    - Strips '#' but keeps the hashtag word
    - Keeps alphabetic tokens and explicit negations ("no", "not", "nor", "n't")
    - Lowercases; optionally lemmatizes
    - Removes stopwords except negations
    - Filters out single-letter noise (except 'i')
    Returns a list of tokens (used by CountVectorizer).
    """

    # Strip URLs and mentions
    text = re.sub(r"http\S+|www\.\S+", " ", text)
    text = re.sub(r"@\w+", " ", text)
    ## YOUR_CODE_STARTS_HERE
    text = re.sub(r"#", " ", text)     # remove just the hashtag
    doc = nlp(text)
    tokens = []
    for token in doc:
      if token.is_alpha or token.text in NEGATIONS:   # keep alphabetic and negation tokens
        if use_lemmas is True:
          lemma = token.lemma_.lower()        #get lemma of eligible token
          if (lemma not in STOP_TW or lemma in NEGATIONS) and (len(lemma) > 1 or lemma == "i"):
            tokens.append(lemma)      # append lemmas that are negations, words with length > 1 and i

    return tokens                # return as list of strings
    ## YOUR_CODE_ENDS_HERE

# Vectorization (Binary Bag-of-Words over uni/bi-grams)
print("Step 2: Vectorize with Binary Bag-of-Words (fit on train only)")

# Stratified Train/Val/Test split: 80/10/10
X_train_text, X_temp_text, y_train, y_temp = train_test_split(
    texts, labels, test_size=0.2, random_state=RANDOM_SEED, stratify=labels
)
X_val_text, X_test_text, y_val, y_test = train_test_split(
    X_temp_text, y_temp, test_size=0.5, random_state=RANDOM_SEED, stratify=y_temp
)

vectorizer = CountVectorizer(
    tokenizer=lambda s: tokenize_tweet(s, use_lemmas=True),
    token_pattern=None,     # needed when using a custom tokenizer
    lowercase=False,        # we already lowercase in the tokenizer
    ngram_range=(1, 2),     # unigrams and bigrams
    min_df=5,               # ignore tokens that don't appear in at least 5 tweets
    binary=True
)

X_train = vectorizer.fit_transform(X_train_text)
X_val   = vectorizer.transform(X_val_text)
X_test  = vectorizer.transform(X_test_text)

print(f"✓ Train shape: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# Convert to dense tensors for PyTorch
X_train_t = torch.tensor(X_train.toarray(), dtype=torch.float32)
X_val_t   = torch.tensor(X_val.toarray(),   dtype=torch.float32)
X_test_t  = torch.tensor(X_test.toarray(),  dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)
y_val_t   = torch.tensor(y_val,   dtype=torch.long)
y_test_t  = torch.tensor(y_test,  dtype=torch.long)

# PyTorch Dataset/DataLoader + MLP
print("Step 3: Build DataLoaders & define MLP")

from torch.utils.data import TensorDataset, DataLoader

train_ds = TensorDataset(X_train_t, y_train_t)
val_ds   = TensorDataset(X_val_t,   y_val_t)
test_ds  = TensorDataset(X_test_t,  y_test_t)

BATCH_SIZE = 128
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE)

class TweetMLP(nn.Module):
    def __init__(self, input_dim: int, hidden: int = 256, dropout: float = 0.2):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden, 2)

    def forward(self, x):
        ## YOUR_CODE_STARTS_HERE
        out = self.fc1(x)          #first linear
        out = self.relu(out)    #relu applied on output
        out = self.drop(out)    #dropout applied on output of relu
        out = self.fc2(out)   #second linear that will return final output
        return out
        ## YOUR_CODE_ENDS_HERE

model = TweetMLP(input_dim=X_train_t.shape[1], hidden=256, dropout=0.1).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

print("✓ Model:")
print(model)

# Training with validation
print("Step 4: Train with validation")

EPOCHS = 10

def run_epoch(dl, train: bool = True) -> Tuple[float, float]:

    ## YOUR_CODE_STARTS_HERE
    for step, (input, output) in enumerate(dl):    # iterate through dataloader input, result pairs

      if train == True:    # only dureing training
        optimizer.zero_grad()

      out = model(input)     # get the output from modelling the input
      loss = criterion(out, output)   # get loss vlaue between predicted and true result

      if train == True:    # only during training
        loss.backward()
        optimizer.step()

      with torch.no_grad():
        _, predicted = torch.max(out, dim=1)
        acc = (predicted == output).sum().item()/len(output)    # calculate accuracy

    return loss, acc
    ## YOUR_CODE_ENDS_HERE

for epoch in range(1, EPOCHS+1):
    tr_loss, tr_acc = run_epoch(train_loader, train=True)
    va_loss, va_acc = run_epoch(val_loader,   train=False)
    print(f"[TASK1] Epoch {epoch:02d} | train loss {tr_loss:.4f} acc {tr_acc:.4f} | val loss {va_loss:.4f} acc {va_acc:.4f}")


# Evaluation
print("Step 5: Test evaluation")

SENT = {0: "neg", 1: "pos"}

model.eval()
all_preds, all_golds = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        logits = model(xb)
        preds = logits.argmax(dim=1).cpu().numpy().tolist()
        all_preds.extend(preds)
        all_golds.extend(yb.numpy().tolist())

acc = accuracy_score(all_golds, all_preds)
cm  = confusion_matrix(all_golds, all_preds)

print(f"[TASK1] ✓ Test Accuracy: {acc:.4f}")
print("[TASK1] Confusion matrix [rows=true 0/1, cols=pred 0/1]:")
print(cm)

# Show 5 random predictions
rng = np.random.default_rng(RANDOM_SEED)
idxs = rng.choice(len(X_test_text), size=5, replace=False)
print("\n[TASK1] Sample predictions:")
for i in idxs:
    gold_lbl = y_test[i]
    pred_lbl = all_preds[i]
    print(f"- Tweet: {X_test_text[i]!r}")
    print(f"  → Gold: {gold_lbl} ({SENT[gold_lbl]})  |  Pred: {pred_lbl} ({SENT[pred_lbl]})")

# RESULT_CHECKING_POINT
print(f"[TASK1 RESULT_CHECKING_POINT] Test Accuracy: {acc:.3f}")

✓ Using device: cpu
Step 1: Load tweets & build tokenizer
✓ Loaded 10000 tweets: 5000 positive, 5000 negative
Step 2: Vectorize with Binary Bag-of-Words (fit on train only)
✓ Train shape: (8000, 1580), Val: (1000, 1580), Test: (1000, 1580)
Step 3: Build DataLoaders & define MLP
✓ Model:
TweetMLP(
  (fc1): Linear(in_features=1580, out_features=256, bias=True)
  (relu): ReLU()
  (drop): Dropout(p=0.1, inplace=False)
  (fc2): Linear(in_features=256, out_features=2, bias=True)
)
Step 4: Train with validation
[TASK1] Epoch 01 | train loss 0.5660 acc 0.7812 | val loss 0.5081 acc 0.7019
[TASK1] Epoch 02 | train loss 0.3785 acc 0.7969 | val loss 0.4522 acc 0.7596
[TASK1] Epoch 03 | train loss 0.4393 acc 0.7812 | val loss 0.4471 acc 0.7500
[TASK1] Epoch 04 | train loss 0.2666 acc 0.9219 | val loss 0.4386 acc 0.7885
[TASK1] Epoch 05 | train loss 0.3103 acc 0.8281 | val loss 0.4501 acc 0.7885
[TASK1] Epoch 06 | train loss 0.2782 acc 0.8750 | val loss 0.4597 acc 0.7981
[TASK1] Epoch 07 | train los

In [ ]:
# ------------------------- TESTS & EXPECTED OUTPUT -------------------------
#
# You should see:
# • Reasonable val accuracy (~0.70–0.75) after a few epochs.
#
# Example (illustrative):
# [TASK1] Epoch 01 | train loss 0.6044 acc 0.69 | val loss 0.5103 acc 0.7540
# ...
# [TASK1] Confusion matrix
# [[375 125]
# [155 345]]
#
# [TASK1] Sample predictions:
# - Tweet: '@oohdawg_ Hi liv :))'
#   → Gold: 1 (pos)  |  Pred: 1 (pos)
# ...
# [TASK1] ✓ Test Accuracy: 0.72 (Expected to be in the range ~0.70–0.75)
# ---------------------------------------------------------------------------

# TASK 2 (5 pts): Transition-based dependency parsing (Arc-Standard)

**High-level overview.**  
Dependency parsing links each word to its **head** forming a tree. The **Arc-Standard** system builds this tree **incrementally** with three moves over a `(STACK, BUFFER, ARCS)` configuration. The core constraint is **projectivity**: you only reduce (remove) a token after attaching all of its dependents, which prevents crossing arcs.

**What you will build.**  
A parser that (1) implements the three transitions, (2) uses a **static oracle** to decide the next legal action from gold heads, and (3) runs a loop that recovers the **gold** projective tree for our toy sentences.

---

### State & actions
- **Initial/terminal states**: start with `STACK=[0]` (ROOT), `BUFFER=[1..n]`, `ARCS=∅`; stop when `BUFFER` is empty and `STACK==[0]`.
- **Let** `s1 = stack[-1]` and `s2 = stack[-2]` (when they exist):
  - `SHIFT`: move the next token from `BUFFER` to the top of `STACK`.
  - `LEFT_ARC`: add `(s1 ← s2)` and remove `s2` (never reduce `ROOT`).
  - `RIGHT_ARC`: add `(s2 ← s1)` and remove `s1`.

---

### What you should implement
1) **Transitions** — `SHIFT`, `LEFT_ARC`, `RIGHT_ARC` that mutate `STACK/BUFFER/ARCS` and guard against invalid moves (e.g., empty `BUFFER`, fewer than two items on `STACK`, never reduce `ROOT`).  
2) **Static oracle** — Using the gold head array:
   - Define **subtree complete**: a node is complete when all its gold dependents are already in `ARCS`.
   - Prefer `LEFT_ARC` if `head(s2)=s1` **and** `s2` is complete; else prefer `RIGHT_ARC` if `head(s1)=s2` **and** `s1` is complete; otherwise `SHIFT`.  
   - If no action is legal with an empty buffer, raise an error (likely non-projective or an earlier mistake).
3) **Parsing loop** — From the initial state, repeatedly query the oracle, apply the action, and on termination convert `ARCS` to a `heads` list with `heads[0] = -1`.

---

### Grading (5 pts)
- 1.1 Transitions (SHIFT/LEFT_ARC/RIGHT_ARC) — **2 pts**  
- 1.2 Static oracle (correct preferences & completeness check) — **2 pts**  
- 1.3 Exact reconstruction on the toy set (UAS = 1.0 for projective cases) — **1 pt**

  → Grading for this point is only for verifying that the **final reported UAS matches the expected correct value (e.g., 1.0 on the toy projective set)**, and **not for performing the evaluation logic itself.**

> **Tips**  
> • A tiny helper for “subtree complete” keeps the oracle clear.  
> • To debug, log `(STACK, BUFFER, action)` for a short sentence.  
> • If you get stuck, check that you didn’t reduce a token before attaching its dependents.


In [24]:

class ArcStandardParser:
    def __init__(self):
        pass

    @staticmethod
    def shift(stack: List[int], buffer: List[int]) -> None:
        ## YOUR_CODE_STARTS_HERE
        stack.append(buffer[0])      # get token from buffer
        buffer.remove(buffer[0])    #remove token from stack
        ## YOUR_CODE_ENDS_HERE

    @staticmethod
    def left_arc(stack: List[int], arcs: Dict[int, int]) -> None:
        """
        Create arc (stack[-1] <- stack[-2]) and remove stack[-2].
        I.e., head = stack[-1], dep = stack[-2]
        """
        ## YOUR_CODE_STARTS_HERE
        arcs[stack[-2]] = stack[-1]     # last element head of second to last element
        stack.remove(stack[-2])
        ## YOUR_CODE_ENDS_HERE

    @staticmethod
    def right_arc(stack: List[int], arcs: Dict[int, int]) -> None:
        """
        Create arc (stack[-2] <- stack[-1]) and remove stack[-1].
        I.e., head = stack[-2], dep = stack[-1]
        """
        ## YOUR_CODE_STARTS_HERE
        arcs[stack[-1]] = stack[-2]   # second to last element head of last element
        stack.remove(stack[-1])
        ## YOUR_CODE_ENDS_HERE

    @staticmethod
    def oracle_action(stack: List[int], buffer: List[int], gold_heads: List[int], built: Dict[int, int]) -> str:
        """
        Return 'LEFT_ARC' | 'RIGHT_ARC' | 'SHIFT' for a projective gold tree.
        Hints:
        - You can LEFT_ARC when stack[-2]'s head is stack[-1] and stack[-2] has no remaining right dependents in buffer/stack.
        - You can RIGHT_ARC when stack[-1]'s head is stack[-2] and stack[-1] has no remaining dependents in buffer.
        - Else SHIFT.
        """
        def subtree_complete(h: int) -> bool:
            # all gold dependents of h have already been attached (in `built`)
            return all((d in built) for d in range(len(gold_heads)) if gold_heads[d] == h)
        ## YOUR_CODE_STARTS_HERE

        if len(stack) < 2:   # put at least 2 elements in stack to start comparing
          return 'SHIFT'

        last_el = stack[-1]     # define elements
        sec_last = stack[-2]

        if gold_heads[last_el] == sec_last and subtree_complete(last_el):     # if dependent and dependency subtree complete
          return 'RIGHT_ARC'
        elif gold_heads[sec_last] == last_el and subtree_complete(sec_last):
          return 'LEFT_ARC'

        return 'SHIFT'
        ## YOUR_CODE_ENDS_HERE

    def parse_oracle(self, sent: DepSentence) -> List[int]:
        stack = [0]
        buffer = list(range(1, len(sent.tokens)))
        arcs: Dict[int, int] = {}   # dep -> head

        ## YOUR_CODE_STARTS_HERE

        gold_heads = sent.heads    # get heads to use in oracle_action

        while(buffer or len(stack) > 1):   # loop until buffer is empty or stack has one element

          action = self.oracle_action(stack, buffer, gold_heads, arcs)    # decide on action based on gold heads

          if action == 'SHIFT':
            if buffer:
              self.shift(stack, buffer)   #if buffer is not empty shift from buffer to stack
            else:     #raise error
              raise ValueError("No action legal. Likely non-projective or an earlier mistake")
          if action == 'RIGHT_ARC':
            self.right_arc(stack, arcs)
          if action == 'LEFT_ARC':
            self.left_arc(stack, arcs)

        heads = [-1] * len(sent.tokens)    # arcs converted to head list
        for dep, head in arcs.items():
          heads[dep] = head      # create entry (dep, head)

        return heads

        ## YOUR_CODE_ENDS_HERE

# TEST_CASES (should be close to UAS=1.0 on projective ones)
parser = ArcStandardParser()
uas_scores = []
for s in CORPUS:
    pred = parser.parse_oracle(s)
    print(pred)
    print(s.heads)

    uas, _ = uas_las(pred, s.heads)
    uas_scores.append(uas)
    print(f"[TASK2 TEST] {s.text} -> UAS={uas:.3f}")
    pretty_tree(s.tokens, pred)

# RESULT_CHECKING_POINT
mean_uas = float(np.mean(uas_scores))
print(f"[TASK2 RESULT_CHECKING_POINT] Mean UAS = {mean_uas:.3f}")


[-1, 4, 4, 4, 5, 0, 5, 9, 9, 6, 5]
[-1, 4, 4, 4, 5, 0, 5, 9, 9, 6, 5]
[TASK2 TEST] The quick brown fox jumps over the lazy dog. -> UAS=1.000
TOK	HEAD
1:The	4:fox
2:quick	4:fox
3:brown	4:fox
4:fox	5:jumps
5:jumps	0:<ROOT>
6:over	5:jumps
7:the	9:dog
8:lazy	9:dog
9:dog	6:over
10:.	5:jumps
[-1, 2, 0, 4, 2, 4, 7, 5, 2]
[-1, 2, 0, 4, 2, 4, 7, 5, 2]
[TASK2 TEST] I saw the man with the telescope. -> UAS=1.000
TOK	HEAD
1:I	2:saw
2:saw	0:<ROOT>
3:the	4:man
4:man	2:saw
5:with	4:man
6:the	7:telescope
7:telescope	5:with
8:.	2:saw
[-1, 2, 0, 2, 2, 6, 4, 2]
[-1, 2, 0, 2, 2, 6, 4, 2]
[TASK2 TEST] Students read books in the library. -> UAS=1.000
TOK	HEAD
1:Students	2:read
2:read	0:<ROOT>
3:books	2:read
4:in	2:read
5:the	6:library
6:library	4:in
7:.	2:read
[-1, 3, 3, 0, 5, 3, 3, 3]
[-1, 3, 3, 0, 5, 3, 3, 3]
[TASK2 TEST] She will finish the report tomorrow. -> UAS=1.000
TOK	HEAD
1:She	3:finish
2:will	3:finish
3:finish	0:<ROOT>
4:the	5:report
5:report	3:finish
6:tomorrow	3:finish
7:.	3:finish
[-1, 2, 4, 4

In [ ]:
# ------------------------- TESTS & EXPECTED OUTPUT -------------------------
#
# For the toy corpus, with spaCy 'en_core_web_sm', the oracle parser should
# exactly reconstruct the gold trees (projective), giving UAS=1.000 per sentence.
#
# You should see lines like:
#
# [TASK2 TEST] The quick brown fox jumps over the lazy dog. -> UAS=1.000
# TOK    HEAD
# 1:The  <index>:<token>
# 2:quick ...
# ...  (pretty_tree prints the head index and token for every word)
#
# [TASK2 TEST] I saw the man with the telescope. -> UAS=1.000
# TOK    HEAD
# ...
# ...
# [TASK2 RESULT_CHECKING_POINT] Mean UAS = 1.000
# ---------------------------------------------------------------------------

# TASK 3 (8 pts): Lesk Word Sense Disambiguation (WSD)

**High-level overview.**  
Word Sense Disambiguation chooses the intended **sense (WordNet synset)** of an ambiguous word from context. The **Lesk** family scores each candidate sense by how many words from its **definition (gloss)** overlap with the **sentence context**. In this task you will build two tightly controlled variants to isolate the impact of **lemmatization**, while keeping everything else fixed and reproducible.

**What you will implement.**  
Two Lesk scorers that both use the **full sentence** as context and identical preprocessing; they differ only in whether they **lemmatize**:
- **Basic Lesk** — surface-form overlap (no lemmatization).  
- **Extended Lesk** — same scoring and context, but with **spaCy lemmatization** of both context and gloss.  
Evaluate both on the provided gold set and report accuracies.

---

## Shared setup (for both variants)
- **Tokenization**: use **spaCy** to tokenize; keep **alphabetic** tokens and lowercase.
- **Stopwords**: use **NLTK** (`nltk.corpus.stopwords`) to remove stopwords.
- **Context scope**: **entire sentence**; remove the target token from the context.
- **POS filtering**: map spaCy coarse POS → WordNet (`NOUN→wn.NOUN`, `VERB→wn.VERB`, `ADJ→wn.ADJ`, `ADV→wn.ADV`) to prune candidates.
- **Scoring & ties**: compare **sets** (unique tokens) via overlap size; break ties by **earliest sense index** (proxy for frequency).

---

## 3.1 (3 pts) Basic Lesk
Build a baseline that:
1) Forms the **context set** from the sentence using spaCy surface tokens (no lemmas) → lowercase, drop NLTK stopwords, remove the target.  
2) Forms the **sense set** from each candidate synset’s **gloss** with the **same** preprocessing (surface forms).  
3) Selects the synset with **maximum set overlap** with the context (tie-break by index).

---

## 3.2 (3 pts) Extended Lesk (with lemmatization)
Repeat 3.1 but **only** change token forms to lemmas:
1) **Context lemmas**: spaCy lemmas → lowercase, drop NLTK stopwords; remove the **lemmatized target**.  
2) **Gloss lemmas**: spaCy lemmas of the synset **gloss** with the same filtering.  
3) Score and tie-break exactly as in Basic.  
> The context is still the **full sentence**; any gain comes from **lemmatization**.

---

## 3.3 (2 pts) Evaluation
- For each gold item, print: variant (**BASIC/EXT**), sentence, target, predicted synset, gold synset, and `OK=True/False`.  
- Report **Accuracy (BASIC)** and **Accuracy (EXT)**.

---

## Grading (8 pts)
- **3.1** Basic Lesk — **3 pts**  
- **3.2** Extended Lesk (same pipeline, **spaCy lemmatization** on context & gloss) — **3 pts**  
- **3.3** Evaluation (accuracies) — **2 pts**

  → Grading for this point is only for verifying that the **final reported accuracies (BASIC and EXT) fall within a reasonable/expected range or value**, and **not for performing the evaluation logic itself.**

> **Tips**  
> • Centralize preprocessing in helpers and toggle only a “use lemmas” flag to keep the variants identical elsewhere.  
> • Remove the target **after** lemmatization in the extended variant.  


In [28]:
import spacy
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from typing import List, Optional

nltk.download('stopwords')
nlp = spacy.load("en_core_web_sm")

# Stopwords: NLTK
from nltk.corpus import stopwords
STOP = set(stopwords.words("english"))

# Tokenization helpers: spaCy
def spacy_tokens(text: str) -> List[str]:
    """
    Tokenize with spaCy and return LOWERCASED surface forms (no lemmas).
    - Keep alphabetic tokens only (drop numbers/punct).
    - Filter using NLTK stop words.
    This is used by *basic* Lesk to avoid lemmatization.
    """
    doc = nlp(text)
    return [t.text.lower() for t in doc if t.is_alpha and t.text.lower() not in STOP]

def spacy_lemmas(text: str) -> List[str]:
    """
    Tokenize + LEMMATIZE with spaCy and return LOWERCASED lemmas.
    - Keep alphabetic lemmas only.
    - Filter using NLTK stop words.
    This is used by *extended* Lesk to show the benefit of lemmatization.
    """
    doc = nlp(text)
    lemmas = []
    for t in doc:
        if not t.is_alpha:
            continue
        lemma = t.lemma_.lower()
        if lemma and lemma.isalpha() and lemma not in STOP:
            lemmas.append(lemma)
    return lemmas

def spacy_lemma_of_token(token_str: str) -> str:
    """
    Lemma of a single token string via spaCy (lowercased).
    Used to remove the target from the lemmatized context in extended Lesk.
    """
    d = nlp(token_str)
    return d[0].lemma_.lower() if len(d) else token_str.lower()

# POS mapping
def wn_pos_from_coarse(coarse: str):
    mappings = {"NOUN": wn.NOUN, "VERB": wn.VERB, "ADJ": wn.ADJ, "ADV": wn.ADV}
    return mappings.get(coarse, None)

# Lesk implementations
def lesk_basic(sentence: str, target: str, coarse_pos: Optional[str] = None) -> Optional[wn.synset]:
    """
    Basic Lesk (NO lemmatization):
      - spaCy tokenization (surface forms)
      - set overlap between sentence tokens and sense gloss
      - tie-break by earliest sense index
    Returns:
      best_synset
    """
    ## YOUR_CODE_STARTS_HERE
    max_lap = 0
    best_synset = None    # initialize for return

    context_set = spacy_tokens(sentence)     # tokenize sentence to get context words
    if target.lower() in context_set:        # remove target from context set
      context_set.remove(target.lower())

    pos = wn_pos_from_coarse(coarse_pos)      # If coarse pos is set
    if coarse_pos:
      sense_set = wn.synsets(target, pos)   # sense set of target word with Parts Of Speech
    else:
      sense_set = wn.synsets(target)

    for sense_t in sense_set:     # for each sense of target
      def_targ = sense_t.definition().lower()    #get definition of sense
      def_set = spacy_tokens(def_targ)           #get tokens from definition

      lap = len(set(context_set) & set(def_set))   # turn to sets and count overlap

      for cont in context_set:
        sense_cont = wn.synsets(cont)    # get sense of each context
        for sense_c in sense_cont:
          def_cont = sense_c.definition().lower()    # get definition of each sense
          def_cont_set = spacy_tokens(def_cont)     # tokenize the definition

          lap += len(set(def_cont_set) & set(def_set))    # count overlap continously
                                                        # between definitions of context words and definition of target
      if lap > max_lap:
        max_lap = lap
        best_synset = sense_t  # set max if definition has more overlaping

        # First sense that reaches maximum gets returned
        # Even if other senses have equal overlap they do not exceed max_lap

    return best_synset
    ## YOUR_CODE_ENDS_HERE


def lesk_extended(sentence: str,
                  target: str,
                  coarse_pos: Optional[str] = None) -> Optional[wn.synset]:
    """
    Extended Lesk (WITH lemmatization):
      - spaCy tokenization + lemmatization everywhere (context & sense bags)
      - set overlap between sentence tokens and sense gloss
      - tie-break by earliest sense index
    Returns:
      best_synset
    """
    ## YOUR_CODE_STARTS_HERE
    max_lap = 0
    best_synset = None           # same structure as basic lesk

    context_set = spacy_lemmas(sentence)         # get sentence lemmas
    token_lemma = spacy_lemma_of_token(target)       # lemmatize the target

    if token_lemma in context_set:
      context_set.remove(token_lemma)    # remove lemma of target from context set

    pos = wn_pos_from_coarse(coarse_pos)
    if coarse_pos:
      sense_set = wn.synsets(target, pos = pos)
    else:
      sense_set = wn.synsets(target)

    for sense_t in sense_set:
      def_targ = sense_t.definition().lower()
      def_set = spacy_lemmas(def_targ)          # lemmatize + tokenize definition of sense candidate

      lap = len(set(context_set) & set(def_set))

      for cont in context_set:
        sense_cont = wn.synsets(cont)       # get sense for each context word
        for sense_c in sense_cont:
          def_cont = sense_c.definition().lower()
          def_cont_set = spacy_lemmas(def_cont)   #lemmatize + tokenize definition of context word

          lap += len(set(def_cont_set) & set(def_set))

      if lap > max_lap:
        max_lap = lap
        best_synset = sense_t

    return best_synset
    ## YOUR_CODE_ENDS_HERE


# Driver that uses FULL sentence for both methods
def disambiguate_with_spacy(sent: DepSentence,
                            target_index: int,
                            extended: bool = True) -> Optional[wn.synset]:
    """
    Resolve the sense of the token at `target_index` in `sent`.
    Returns:
      (best_synset, best_overlap_count)
    """
    target = sent.tokens[target_index]
    coarse_pos = sent.pos[target_index]

    if extended:
        return lesk_extended(sent.text, target, coarse_pos=coarse_pos)
    else:
        return lesk_basic(sent.text, target, coarse_pos=coarse_pos)


# Evaluation
def eval_wsd(extended: bool = False) -> float:
    correct = 0
    for sent, idx, gold in GOLD_WSD:
        syn = disambiguate_with_spacy(sent, idx, extended=extended)
        got = syn.name() if syn else None
        acc = int(got == gold)
        correct += int(acc)
        print(f"[TASK3 EVAL] {'EXT' if extended else 'BASIC'} \t sentence={sent.text}\t target={sent.tokens[idx]} "
              f"\tgot={got} \tgold={gold} \taccuracy={acc}")
    n = len(GOLD_WSD)
    avg_acc = correct / n if n else 0.0
    print(f"Avg accuracy ({'EXT' if extended else 'BASIC'}): {avg_acc:.3f}")
    return avg_acc

# --- Gold set & evaluation ---
GOLD_WSD = [
    # bank
    (spacy_to_dep("He went to the bank to do few transactions."), 5, "bank.n.09"),
    (spacy_to_dep("We sat on the bank of the river."),               5, "bank.n.01"),
    # bass
    (spacy_to_dep("The musician tuned his bass instrument before the show."),   5, "bass.n.07"),
    (spacy_to_dep("They caught a large bass in the lakes."),          5, "bass.n.08"),
    # plant
    (spacy_to_dep("The chemical plant employs 200 labor workers."),        3, "plant.n.01"),
    (spacy_to_dep("Please water the plant by the window."),          4, "plant.n.02"),
    # file
    (spacy_to_dep("Place the file record on your desk."),                 3, "file.n.01"),
    (spacy_to_dep("They will file complaints to the office tomorrow."),           3, "file.v.01"),
    # interest
    (spacy_to_dep("The banks raised the percentage of interest on loans."),             7, "interest.n.04"),
    (spacy_to_dep("Science fiction is her main interest."),          6, "interest.n.01"),

]

# --- TEST_CASES ---
acc_basic = eval_wsd(extended=False)
acc_ext   = eval_wsd(extended=True)
print("[TASK3 RESULT_CHECKING_POINT] "
      f"Avg accuracy BASIC vs EXT: {acc_basic:.3f} {acc_ext:.3f}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[TASK3 EVAL] BASIC 	 sentence=He went to the bank to do few transactions.	 target=bank 	got=depository_financial_institution.n.01 	gold=bank.n.09 	accuracy=0
[TASK3 EVAL] BASIC 	 sentence=We sat on the bank of the river.	 target=bank 	got=bank.n.01 	gold=bank.n.01 	accuracy=1
[TASK3 EVAL] BASIC 	 sentence=The musician tuned his bass instrument before the show.	 target=bass 	got=bass.n.07 	gold=bass.n.07 	accuracy=1
[TASK3 EVAL] BASIC 	 sentence=They caught a large bass in the lakes.	 target=bass 	got=bass.n.01 	gold=bass.n.08 	accuracy=0
[TASK3 EVAL] BASIC 	 sentence=The chemical plant employs 200 labor workers.	 target=plant 	got=plant.n.01 	gold=plant.n.01 	accuracy=1
[TASK3 EVAL] BASIC 	 sentence=Please water the plant by the window.	 target=plant 	got=plant.n.04 	gold=plant.n.02 	accuracy=0
[TASK3 EVAL] BASIC 	 sentence=Place the file record on your desk.	 target=file 	got=file.n.01 	gold=file.n.01 	accuracy=1
[TASK3 EVAL] BASIC 	 sentence=They will file complaints to the office to

In [ ]:
# -------------------- TESTS & EXPECTED OUTPUT --------------------
#
# The evaluation prints one line per item, then avg accuracy lines for BASIC and EXT.
# Because results can vary slightly with stopword source and WordNet version,
# focus on this invariant:
#   • Extended (windowed) accuracy should be >= Basic accuracy
#   • On this toy set, Extended and Basic accuracies should be 0.600 and 0.700
#
# Example format (your synset names may vary on some items):
# [TASK3 EVAL] BASIC  He deposited cash at the bank near his office.  target=bank  got=bank.n.09  gold=bank.n.09  accuracy=0
# ...
# Avg accuracy (BASIC): 0.600
# [TASK3 EVAL] EXT    He deposited cash at the bank near his office.  target=bank  got=bank.n.09  gold=bank.n.09  accuracy=0
# ...
# Avg accuracy (EXT): 0.700
# [TASK3 RESULT_CHECKING_POINT] Acc basic vs ext: 0.600 0.700
# -----------------------------------------------------------------

# BONUS (2 pts): Solve any task with an LLM

**Goal.**  
Pick **one** of the homework tasks (Task 1, Task 2, or Task 3) and solve it using an **LLM**.  
Then **document** what you did and **compare** the LLM’s result to your own pipeline.

**What to deliver below.**
1) **LLM used** (name + version, e.g., “Llama-3-8B-Instruct”, “GPT-x”, “Claude-x”, “Mistral-x”, etc.).  
2) **Prompt(s)** you used.  
3) **LLM output** — copy and paste it.  
4) **Comparison** to your solution: what matches or differs (quantitative or qualitative).  
5) **Reflection**: what the LLM was **good at** vs **bad at**, what it got **right** vs **wrong**.

> **No code required.** You do **not** need to run, share, or submit any code used for the LLM generation. Provide only the deliverables listed above.
> You may use any LLMs through any interface (API, web UI, local inference).
"""


# BONUS (2 pts): Solve any task with an LLM

**Goal:** Solve Task 2 using a LLM

1) **LLM used:** GPT-5

2) **Prompt(s):**  

    For the following sentences perform transition-based dependency parsing (arc-standard). By implementing three transitions: shift, left arc and right arc over a buffer a stack, store dependencies as arcs. A subtree is complete when all of a node's dependencies are already in arcs. As results produce the projective dependency trees of the sentences.

    Explain the steps for one sentence, as only produce outputs for the rest.


    1. "The quick brown fox jumps over the lazy dog.",

    2. "I saw the man with the telescope.",

    3. "Students read books in the library.",

    4. "She will finish the report tomorrow.",

    5. "Flying planes can be dangerous."

3) **LLM output**

    🔹 Arc-Standard Transition System Recap

    Configuration:

    Stack (S),
    Buffer (B),
    Set of arcs (A)

    Transitions:

    SHIFT: Move the first word from buffer to stack.
    ([S], [w_i | B], A) → ([S | w_i], B, A)

    LEFT-ARC(l): Add arc (S[1] ← S[0]) and remove S[1] (the head keeps its dependent).
    ([S | w_i | w_j], B, A) → ([S | w_j], B, A ∪ {(w_j, l, w_i)})

    RIGHT-ARC(l): Add arc (S[1] → S[0]) and remove S[0].
    ([S | w_i | w_j], B, A) → ([S | w_i], B, A ∪ {(w_i, l, w_j)})

    We stop when the buffer is empty and only the root remains on the stack.

    🦊 Sentence 1: “The quick brown fox jumps over the lazy dog.”

    Let’s assume “jumps” is the ROOT.


    | Step | Stack               | Buffer                                                | Action          | New Arc         |
    | ---- | ------------------- | ----------------------------------------------------- | --------------- | --------------- |
    | 0    | []                  | [The, quick, brown, fox, jumps, over, the, lazy, dog] | INIT            | —               |
    | 1    | [The]               | [quick, brown, fox, jumps, over, the, lazy, dog]      | SHIFT           | —               |
    | 2    | [The, quick]        | [brown, fox, jumps, over, the, lazy, dog]             | LEFT-ARC(det)   | (quick ← The)   |
    | 3    | [quick]             | [brown, fox, jumps, over, the, lazy, dog]             | SHIFT           | —               |
    | 4    | [quick, brown]      | [fox, jumps, over, the, lazy, dog]                    | LEFT-ARC(amod)  | (brown ← quick) |
    | 5    | [brown]             | [fox, jumps, over, the, lazy, dog]                    | SHIFT           | —               |
    | 6    | [brown, fox]        | [jumps, over, the, lazy, dog]                         | LEFT-ARC(nsubj) | (jumps ← fox)   |
    | 7    | [jumps]             | [over, the, lazy, dog]                                | SHIFT           | —               |
    | 8    | [jumps, over]       | [the, lazy, dog]                                      | SHIFT           | —               |
    | 9    | [jumps, over, the]  | [lazy, dog]                                           | LEFT-ARC(det)   | (the ← over)    |
    | 10   | [jumps, over]       | [lazy, dog]                                           | SHIFT           | —               |
    | 11   | [jumps, over, lazy] | [dog]                                                 | LEFT-ARC(amod)  | (lazy ← over)   |
    | 12   | [jumps, over]       | [dog]                                                 | SHIFT           | —               |
    | 13   | [jumps, over, dog]  | []                                                    | LEFT-ARC(pobj)  | (over ← dog)    |
    | 14   | [jumps, over]       | []                                                    | RIGHT-ARC(prep) | (jumps → over)  |
    | 15   | [jumps]             | []                                                    | —               | root complete ✅ |


    Final Arcs (Dependency Tree):

    jumps (ROOT)
    ├── fox (nsubj)
    │     ├── quick (amod)
    │     │     └── The (det)
    │     └── brown (amod)
    └── over (prep)
          └── dog (pobj)
                ├── the (det)
                └── lazy (amod)

    📘 Sentence 2: “I saw the man with the telescope.”

    ROOT: saw

    Arcs:

    saw (ROOT)
    ├── I (nsubj)
    └── man (dobj)
          ├── the (det)
          └── with (prep)
                └── telescope (pobj)
                      └── the (det)

    📗 Sentence 3: “Students read books in the library.”

    ROOT: read

    Arcs:

    read (ROOT)
    ├── Students (nsubj)
    └── books (dobj)
          ├── in (prep)
          │     └── library (pobj)
          │           └── the (det)

    📙 Sentence 4: “She will finish the report tomorrow.”

    ROOT: finish

    Arcs:

    finish (ROOT)
    ├── She (nsubj)
    ├── will (aux)
    ├── report (dobj)
    │     └── the (det)
    └── tomorrow (nmod:tmod)

    📒 Sentence 5: “Flying planes can be dangerous.”

    ROOT: dangerous

    Arcs:

    dangerous (ROOT)
    └── be (cop)
          ├── can (aux)
          └── planes (nsubj)
                └── Flying (amod)



4) **Comparison** to your solution:

    The logic used by the LLM is correctly derived by the prompt.

    Starting by the results for the first sentence, the LLM correctly puts 'jumps' as root, stores three dependencies at 'fox' and three at 'over' (with a subtree of two dependencies at 'dog')

    Sentence 2 and 4 also follow the same logic and produce the same dependency trees. However, differences are shown on sentence 3 and 5.

    In "Students read books in the library", while our model produces a dependency arc between 'in' and the root 'read', the LLM puts 'books' as head of 'in'.

    This means that our model understands the sentance as: *Students read books while being at the library.* while the LLM understands: *Students read books that are in the library.*

    A bigger difference is observed at "Flying planes can be dangerous." While our model puts the verb "be" as root and "dangerous" as dependant on the root, the LLM decides on "dangerous" as root and "be" as the head of the subtree that depends on root.


5) **Reflection**:
    Due to the results on the previous section, it can be observed that the LLM correctly follows the model, with minor differences in the dependency decisions.

    However, these decisions, cannot be concluded as errors or mistakes. Because the sentences can contain ambiguous phrases, both models are prone to interpreting them differently, but that doesn't mean that the dependency tree comes from syntactical errors as both interpretations are correct.

    In the case where dangerous is on the place of the root, it shows that the LLM values core meaning sof the sentences more than formal gramatical rules. The latter are followed by our model putting the verb as the root of the tree.

    As a result, even though both models produce correct parsing dependencies, the LLM is more prone to meaning-focused interpretations than model rules. This can be due to its constant experience with reading and understanding human-written texts directly fromt he user.



"""


# End of HW 01